In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from prophet import Prophet


<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
data = pd.read_csv("../../data/od_15380_jml_pendapatan_asli_drh_bidang_pariwisata__sektor_wisa/data.jabarprov.go.id/disparbud-od_15380_jml_pendapatan_asli_drh_bidang_pariwisata__sektor_wisa_data.csv")
data.head(5)


,id,kode_provinsi,nama_provinsi,kode_kabupaten_kota,nama_kabupaten_kota,sektor_wisata,jumlah_pendapatan,satuan,tahun
0,1,32,JAWA BARAT,3201,KABUPATEN BOGOR,HOTEL,0,RUPIAH,2014
1,2,32,JAWA BARAT,3201,KABUPATEN BOGOR,RESTORAN/RUMAH MAKAN,27946638075,RUPIAH,2014
2,3,32,JAWA BARAT,3201,KABUPATEN BOGOR,HIBURAN,7077430815,RUPIAH,2014
3,4,32,JAWA BARAT,3201,KABUPATEN BOGOR,RETRIBUSI,485255100,RUPIAH,2014
4,5,32,JAWA BARAT,3202,KABUPATEN SUKABUMI,HOTEL,0,RUPIAH,2014


In [14]:
hotel_pendapatan = pd.DataFrame()
hotel_pendapatan["tahun"] = data["tahun"].unique()
hotel_pendapatan["jumlah_pendapatan"] = data[data["sektor_wisata"]
                                                        == "HOTEL"].reset_index()["jumlah_pendapatan"]
hotel_pendapatan = hotel_pendapatan.reset_index().rename(
    columns={"tahun": "ds", "jumlah_pendapatan": "y"}).drop(columns="index")

rm_pendapatan = pd.DataFrame()
rm_pendapatan["tahun"] = data["tahun"].unique()
rm_pendapatan["jumlah_pendapatan"] = data[data["sektor_wisata"]
                                                        == "RESTORAN/RUMAH MAKAN"].reset_index()["jumlah_pendapatan"]
rm_pendapatan = rm_pendapatan.reset_index().rename(
    columns={"tahun": "ds", "jumlah_pendapatan": "y"}).drop(columns="index")

hiburan_pendapatan = pd.DataFrame()
hiburan_pendapatan["tahun"] = data["tahun"].unique()
hiburan_pendapatan["jumlah_pendapatan"] = data[data["sektor_wisata"]
                                                          == "HIBURAN"].reset_index()["jumlah_pendapatan"]
hiburan_pendapatan = hiburan_pendapatan.reset_index().rename(
    columns={"tahun": "ds", "jumlah_pendapatan": "y"}).drop(columns="index")

In [15]:
hiburan_model = Prophet(seasonality_mode="multiplicative")
hiburan_model.fit(hiburan_pendapatan)

hotel_model = Prophet(seasonality_mode="multiplicative")
hotel_model.fit(hotel_pendapatan)

rm_model = Prophet(seasonality_mode="multiplicative")
rm_model.fit(rm_pendapatan)

INFO:cmdstanpy:Chain [1] start processing
INFO:cmdstanpy:Chain [1] done processing
INFO:cmdstanpy:Chain [1] start processing
INFO:cmdstanpy:Chain [1] done processing
INFO:cmdstanpy:Chain [1] start processing
INFO:cmdstanpy:Chain [1] done processing


In [16]:
hiburan_future = pd.DataFrame({
    "ds": [max(hiburan_pendapatan["ds"]) + i for i in range(1, 6)]
})
hiburan_future["y"] = hiburan_model.predict(hiburan_future)["yhat"]
hiburan_gabung = pd.concat([hiburan_pendapatan, hiburan_future])
hiburan_gabung = hiburan_gabung.rename(
    columns={"ds": "tahun", "y": "pendapatan"})

hotel_future = pd.DataFrame({
    "ds": [max(hotel_pendapatan["ds"]) + i for i in range(1, 6)]
})
hotel_future["y"] = hotel_model.predict(hotel_future)["yhat"]
hotel_gabung = pd.concat([hotel_pendapatan, hotel_future])
hotel_gabung = hotel_gabung.rename(columns={"ds": "tahun", "y": "pendapatan"})

rm_future = pd.DataFrame({
    "ds": [max(rm_pendapatan["ds"]) + i for i in range(1, 6)]
})
rm_future["y"] = rm_model.predict(rm_future)["yhat"]
rm_gabung = pd.concat([rm_pendapatan, rm_future])
rm_gabung = rm_gabung.rename(columns={"ds": "tahun", "y": "pendapatan"})


In [18]:
hiburan_gabung.to_csv("hiburan_gabung.csv", index=False)
hotel_gabung.to_csv("hotel_gabung.csv", index=False)
rm_gabung.to_csv("rm_gabung.csv", index=False)